In [18]:
import cv2
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import pandas as pd

# ===================== ПАРАМЕТРЫ =====================
SOURCE_DIR = "/Users/umanindaniil/balakin-2025/task-4 (9 задача)/source_images"      # Папка с готовыми обрезанными символами
NOISY_DIR  = "noisy_symbols"        # Папка для зашумлённых версий
BIN_DIR    = "binary_symbols"       # Папка для бинаризованных версий
MASK_DIR   = "masks"                # Папка для масок (0/1)
SIGMA      = 2500                   # Стандартное отклонение для гауссова шума
THRESH     = 128                    # Порог для бинаризации

# Создаём выходные папки, если их ещё нет
os.makedirs(NOISY_DIR, exist_ok=True)
os.makedirs(BIN_DIR, exist_ok=True)
os.makedirs(MASK_DIR, exist_ok=True)

# ===================== 1. ЧТЕНИЕ ИСХОДНЫХ ФАЙЛОВ =====================
valid_ext = (".jpeg", ".jpg", ".png")
file_list = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(valid_ext)]
file_list.sort()  # Предполагаем, что файлы названы по меткам, например: A.png, B.png и т.д.
if not file_list:
    raise FileNotFoundError(f"Не найдено ни одного изображения в {SOURCE_DIR}")

# Списки для хранения путей и меток
noisy_paths = []    # Пути к зашумлённым изображениям
mask_paths  = []    # Пути к маскам
symbols_names = []  # Истинные метки символов (из названий файлов)

# ===================== 2. ФУНКЦИИ ДЛЯ ОБРАБОТКИ =====================

def add_gaussian_noise(img, sigma=2500):
    """
    Добавляет Гауссов шум со средним 0 и стандартным отклонением sigma.
    img должен быть формата uint8.
    """
    float_img = img.astype(np.float32)
    noise = np.random.normal(0, sigma, size=img.shape).astype(np.float32)
    noisy_float = float_img + noise
    noisy_float = np.clip(noisy_float, 0, 255)
    return noisy_float.astype(np.uint8)

def projector_value(noisy_img, mask_img):
    """
    Вычисляет среднее значение интенсивности зашумлённого изображения
    только по тем пикселям, где mask_img = 255.
    Если размеры не совпадают, маска масштабируется до размеров noisy_img.
    """
    if noisy_img.shape != mask_img.shape:
        mask_img = cv2.resize(mask_img, (noisy_img.shape[1], noisy_img.shape[0]),
                              interpolation=cv2.INTER_NEAREST)
    noisy_f = noisy_img.astype(np.float32)
    mask_f  = (mask_img.astype(np.float32) / 255.0)
    sum_val = np.sum(noisy_f * mask_f)
    count_val = np.sum(mask_f)
    if count_val < 1e-5:
        return 0.0
    return sum_val / count_val

def make_mask_from_binary(bin_img):
    """
    Преобразует бинарное изображение (0/255) в маску (0/1), а затем умножает на 255.
    """
    mask_01 = (bin_img // 255).astype(np.uint8)
    mask_255 = (mask_01 * 255).astype(np.uint8)
    return mask_255

# ===================== 3. ОБРАБОТКА КАЖДОГО ФАЙЛА =====================
for i, filename in enumerate(file_list):
    name_only, ext = os.path.splitext(filename)
    symbols_names.append(name_only)  # Истинная метка символа (например, "A", "B", "C" и т.д.)

    src_path = os.path.join(SOURCE_DIR, filename)
    img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Не удалось открыть {src_path}, пропускаем...")
        continue

    # ---------- (1) Наложение Гауссова шума ----------
    noisy = add_gaussian_noise(img, sigma=SIGMA)
    noisy_path = os.path.join(NOISY_DIR, f"{name_only}_noisy.png")
    cv2.imwrite(noisy_path, noisy)
    noisy_paths.append(noisy_path)

    # ---------- (2) Бинаризация ----------
    # Используем адаптивный порог с инверсией для выделения символа
    bin_img = cv2.adaptiveThreshold(img, 255, 
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                    cv2.THRESH_BINARY_INV, 
                                    11, 2)
    
    # ---------- (3) Морфологическая обработка ----------
    kernel = np.ones((3, 3), np.uint8)
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, kernel, iterations=1)
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel, iterations=1)
    
    bin_path = os.path.join(BIN_DIR, f"{name_only}_bin.png")
    cv2.imwrite(bin_path, bin_img)

    # ---------- (4) Создание маски ----------
    mask_255 = make_mask_from_binary(bin_img)
    mask_path = os.path.join(MASK_DIR, f"{name_only}_mask.png")
    cv2.imwrite(mask_path, mask_255)
    mask_paths.append(mask_path)

# ===================== 4. РАСПОЗНАВАНИЕ =====================
# Сохраним истинные метки и предсказанные для вычисления матрицы путаницы
y_true = []  # Истинные метки
y_pred = []  # Предсказанные метки

for i, npath in enumerate(noisy_paths):
    noisy_img = cv2.imread(npath, cv2.IMREAD_GRAYSCALE)
    if noisy_img is None:
        continue

    current_symbol_name = symbols_names[i]
    results = []
    for j, mp in enumerate(mask_paths):
        m_img = cv2.imread(mp, cv2.IMREAD_GRAYSCALE)
        if m_img is None:
            continue
        val = projector_value(noisy_img, m_img)
        results.append((j, val, symbols_names[j]))

    if results:
        best_match = min(results, key=lambda x: x[1])
        best_idx, best_val, best_symbol = best_match
        print(f"Зашумлённый символ '{current_symbol_name}' наиболее похож на '{best_symbol}' (проектор={best_val:.2f})")
        y_true.append(current_symbol_name)
        y_pred.append(best_symbol)
    else:
        print(f"Не удалось распознать символ '{current_symbol_name}'.")
        y_true.append(current_symbol_name)
        y_pred.append("None")

# ===================== 5. ВЫЧИСЛЕНИЕ МАТРИЦЫ ПУТАНИЦ =====================
# Получаем список уникальных меток (сортированный для удобства)
labels = sorted(list(set(symbols_names)))
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Выводим матрицу путаницы с использованием pandas DataFrame
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
print("\nМатрица путаницы:")
print(df_cm)


Зашумлённый символ 'A' наиболее похож на 'A' (проектор=123.77)
Зашумлённый символ 'B' наиболее похож на 'E' (проектор=129.30)
Зашумлённый символ 'C' наиболее похож на 'C' (проектор=129.08)
Зашумлённый символ 'D' наиболее похож на 'D' (проектор=128.49)
Зашумлённый символ 'E' наиболее похож на 'E' (проектор=125.82)
Зашумлённый символ 'G' наиболее похож на 'H' (проектор=129.15)
Зашумлённый символ 'H' наиболее похож на 'E' (проектор=125.57)
Зашумлённый символ 'I' наиболее похож на 'I' (проектор=126.09)

Матрица путаницы:
   A  B  C  D  E  G  H  I
A  1  0  0  0  0  0  0  0
B  0  0  0  0  1  0  0  0
C  0  0  1  0  0  0  0  0
D  0  0  0  1  0  0  0  0
E  0  0  0  0  1  0  0  0
G  0  0  0  0  0  0  1  0
H  0  0  0  0  1  0  0  0
I  0  0  0  0  0  0  0  1
